# Emerging Technologies – Problems

**Author:** Hammad Mubarik  
**Module:** Emerging Technologies  
**Year:** 2025/2026

## Problem 1: Generating Random Boolean Functions

The [Deutsch–Jozsa algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms#deutschs-algorithm) is a quantum algorithm that figures out whether a given function is constant or balanced in one query. A classical computer would need to check up to half the inputs before it could be sure.

To test the algorithm we need to be able to generate these functions ourselves. The goal here is to write a function called `random_constant_balanced` that returns a random function taking four Boolean inputs and returning a single Boolean output, where the returned function is guaranteed to be either constant or balanced.

- **Constant** means the function returns the same value no matter what you pass in.
- **Balanced** means it returns `True` for exactly half the possible inputs and `False` for the other half.

In [ ]:
# Random selections.
import random

# Numerical arrays and operations.
import numpy as np

# Generating all combinations of inputs.
import itertools as it

### Representing Boolean Functions as Lookup Tables

With four Boolean inputs there are $2^4 = 16$ possible input combinations. Rather than writing out the function logic explicitly, I'm representing each function as a lookup table — a tuple of 16 output values where the position corresponds to the input.

To use it, the four inputs $(a, b, c, d)$ get converted to an integer index and we just read off the value at that position. So for example $(False, True, False, True)$ becomes `0101` in binary which is 5, so we look at index 5 in the table.

This makes it straightforward to control whether the function is constant or balanced when building the table, and the table gets captured in a [closure](https://realpython.com/python-closures/) so the returned function carries it around internally.

In [ ]:
# Show all 16 possible input combinations for four Boolean inputs.
# itertools.product generates the cartesian product of the input iterables.
# See: https://docs.python.org/3/library/itertools.html#itertools.product
all_inputs = list(it.product([False, True], repeat=4))

# Show the first few and the total count.
print(f'Total input combinations: {len(all_inputs)}')
print('First four inputs:', all_inputs[:4])
print('Last  four inputs:', all_inputs[-4:])

### Converting Boolean Inputs to an Index

Each of the four inputs is treated as a bit — `True` is `1` and `False` is `0`. Putting them together gives a 4-bit binary number which we convert to a decimal integer to use as the index.

So $(True, False, True, True)$ becomes $1011_2 = 8 + 0 + 2 + 1 = 11$.

Python's built-in [`int`](https://docs.python.org/3/library/functions.html#int) function handles the binary string to integer conversion directly with base 2.

Here is a quick reference table showing decimal 0–20 and their binary equivalents, which is useful for understanding how the inputs map to positions in the lookup table.

In [ ]:
# Display a decimal to binary reference table for 0 to 20.
# bin() returns a string like '0b1011'; [2:] strips the '0b' prefix.
# See: https://docs.python.org/3/library/functions.html#bin
print(f'{"Decimal":>10} | {"Binary":>8}')
print('-' * 22)
for n in range(21):
    print(f'{n:>10} | {bin(n)[2:]:>8}')

In [ ]:
def bool_args_to_index(a, b, c, d):
    """Convert four Boolean arguments to an integer index.
    
    Each argument is treated as a bit (1 if true, 0 if false).
    The four bits are joined into a binary string and converted to an int.
    
    Example: (True, False, True, True) -> '1011' -> 11
    """
    # Map each argument to '1' or '0'.
    bits = ''.join('1' if x else '0' for x in (a, b, c, d))
    # Convert the binary string to an integer using base 2.
    return int(bits, 2)

In [ ]:
# Quick check: verify the index for a known input.
# (True, False, True, True) is binary 1011, which equals 11.
print(bool_args_to_index(True, False, True, True))

# (False, False, False, False) is binary 0000, which equals 0.
print(bool_args_to_index(False, False, False, False))

# (True, True, True, True) is binary 1111, which equals 15.
print(bool_args_to_index(True, True, True, True))

### Building the Lookup Table

For a **constant** function all 16 entries in the table are the same — either all `False` or all `True`. There are only 2 functions like this.

For a **balanced** function exactly 8 entries are `True` and 8 are `False`. The number of ways to arrange this is $\binom{16}{8} = 12{,}870$, so there is a lot of variety. I build the table as a sorted list first (8 `False` then 8 `True`) and then use [`random.shuffle`](https://docs.python.org/3/library/random.html#random.shuffle) to randomly rearrange it.

The type is picked with equal probability — 50% chance of constant, 50% balanced.

In [ ]:
def random_constant_balanced():
    """Return a randomly chosen constant or balanced function of four Boolean inputs.

    The returned function accepts exactly four Boolean arguments and returns
    a single Boolean value. It is guaranteed to be either:
      - Constant: returns the same value for all 16 possible inputs, or
      - Balanced: returns True for exactly 8 of the 16 possible inputs.

    The function type (constant or balanced) is chosen with equal probability.
    Within each type, the specific function is also chosen uniformly at random.
    """
    # There are 2^4 = 16 possible inputs for four Boolean arguments.
    num_inputs = 2 ** 4

    # Randomly choose whether the function is constant or balanced.
    is_constant = random.choice([True, False])

    if is_constant:
        # Constant: every entry in the lookup table is the same value.
        # Choose randomly between always-False and always-True.
        value = random.choice([False, True])
        lookup = (value,) * num_inputs
    else:
        # Balanced: exactly half the entries are True, half are False.
        # Start with 8 False values followed by 8 True values.
        lookup = [False] * (num_inputs // 2) + [True] * (num_inputs // 2)
        # Shuffle so the True values appear in random positions.
        random.shuffle(lookup)
        # Convert to a tuple so the table is immutable.
        lookup = tuple(lookup)

    def f(a, b, c, d):
        """Take four Boolean inputs and return a Boolean output."""
        # Convert the inputs to an index into the lookup table.
        idx = bool_args_to_index(a, b, c, d)
        # Return the value stored at that index.
        return lookup[idx]

    # Return the closure — the lookup table is captured inside f.
    return f